<a href="https://colab.research.google.com/github/jibitesh2004/jibitesh2004/blob/main/Predict_rocket_launch_delays_with_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT LIBRARIES

In [11]:
import pandas as pd
import numpy as np
import requests
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, Audio
from gtts import gTTS

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

# Install gTTS if needed (silent installation)
!pip install gTTS --quiet

# Set random seed for reproducibility
np.random.seed(42)
print("Setup complete.")


Setup complete.


# **Load Historical Excel Data (Past Data)**


In [13]:
# Try to load the Excel file; if not, generate synthetic data.
try:
    launch_data = pd.read_excel('Generated.xlsx')
    launch_data.columns = [col.strip() for col in launch_data.columns]
    print("Excel data loaded successfully.")
except Exception as e:
    print("Error loading Excel file; using synthetic data. Error:", e)
    launch_data = pd.DataFrame({
        'Name': ['Rocket A'] * 10,
        'Date': pd.date_range(start='2025-01-01', periods=10),
        'Time (East Coast)': ['10:00 AM'] * 10,
        'Location': ['Cape Canaveral'] * 10,
        'Crewed or Uncrewed': np.random.choice(['Crewed', 'Uncrewed'], 10),
        'Launched?': np.random.choice(['Y', 'N'], 10),
        'Wind Direction': np.random.choice(['N', 'S', 'E', 'W'], 10),
        'Condition': np.random.choice(['Fair', 'Cloudy', 'Stormy'], 10)
    })

launch_data.fillna({'Launched?': 'N', 'Crewed or Uncrewed': 'Uncrewed',
                    'Wind Direction': 'unknown', 'Condition': 'Fair'}, inplace=True)

# Synthetic additional features
launch_data['DelayTime'] = np.random.uniform(10, 60, launch_data.shape[0])
launch_data['Temperature'] = np.random.uniform(15, 35, launch_data.shape[0])
launch_data['WindSpeed'] = np.random.uniform(0, 50, launch_data.shape[0])
launch_data['Precipitation'] = np.random.uniform(0, 5, launch_data.shape[0])
launch_data['SensorReading'] = np.random.uniform(0.8, 1.0, launch_data.shape[0])
launch_data['CommLog'] = np.random.choice([
    "All systems nominal, ready for launch.",
    "Minor issues detected, proceed with caution.",
    "Crew on standby, monitoring weather.",
    "Unexpected alert in telemetry, investigate further."
], launch_data.shape[0])
print("Data loaded and preprocessed.")


Excel data loaded successfully.
Data loaded and preprocessed.


# **Clean & Process Historical Data**

In [14]:
# Set target and features
y = launch_data['DelayTime']
cols_to_drop = ['Name', 'Date', 'Time (East Coast)', 'Location', 'Launched?', 'DelayTime']
X = launch_data.drop(columns=cols_to_drop)

# Encode categorical features
le = LabelEncoder()
for col in ['Crewed or Uncrewed', 'Wind Direction', 'Condition']:
    X[col] = le.fit_transform(X[col])

numeric_features = ['Temperature', 'WindSpeed', 'Precipitation', 'SensorReading',
                      'Crewed or Uncrewed', 'Wind Direction', 'Condition']
text_feature = 'CommLog'

# Pipelines for numeric and text features
num_pipe = Pipeline([('scaler', StandardScaler())])
text_pipe = Pipeline([('tfidf', TfidfVectorizer(max_features=50))])
preprocessor = ColumnTransformer([
    ('num', num_pipe, numeric_features),
    ('text', text_pipe, text_feature)
])

# Complete pipeline with a decision tree regressor
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=0, max_depth=5))
])

# Split data and train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)
model_pipeline.fit(X_train, y_train)
score = model_pipeline.score(X_test, y_test)
print("Model trained. R² Score:", round(score, 2))


Model trained. R² Score: nan


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


# **Enrich Historical Data with Additional Features**

In [16]:
def fetch_live_weather():
    """Fetch live weather from Open-Meteo API (Temperature in °C)."""
    try:
        latitude, longitude = 28.3922, -80.6077
        url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
        response = requests.get(url, timeout=5)
        data = response.json()
        cw = data.get("current_weather", {})
        temperature = cw.get("temperature", np.random.uniform(15,35))
        wind_speed = cw.get("windspeed", np.random.uniform(0,50))
        precipitation = 0
        return temperature, wind_speed, precipitation
    except Exception as e:
        print("Error fetching live weather:", e)
        return np.random.uniform(15,35), np.random.uniform(0,50), np.random.uniform(0,5)

def simulate_space_condition():
    return np.random.choice(["Calm", "Moderate", "Disturbed"])

def generate_sensor_alert(sensor_value, threshold=0.9):
    alert = "Alert: Sensor low. " if sensor_value < threshold else "Normal: Sensor OK. "
    alert += f"Space condition: {simulate_space_condition()}."
    return alert

def fetch_space_weather():
    """Fetch space weather data using NASA's DONKI API (demo key)."""
    try:
        base_url = "https://api.nasa.gov/DONKI/FLR"
        params = {"api_key": "DEMO_KEY"}
        response = requests.get(base_url, params=params, timeout=5)
        events = response.json()
        if events and isinstance(events, list):
            event = events[0]
            return f"FLR on {event.get('beginTime', 'Unknown')}"
        else:
            return "No significant space weather events."
    except Exception as e:
        print("Error fetching space weather:", e)
        return "Space weather data unavailable."

def generate_delay_statement(input_data, delay):
    reasons = []
    if input_data['WindSpeed'] > 30:
        reasons.append("high wind speeds")
    if input_data['Precipitation'] > 2:
        reasons.append("heavy precipitation")
    if input_data['SensorReading'] < 0.9:
        reasons.append("low sensor reading")
    if "alert" in input_data['CommLog'].lower():
        reasons.append("sensor alert")
    if not reasons:
        reasons.append("all parameters nominal")
    return f"Predicted delay: {delay:.1f} minutes due to " + ", ".join(reasons) + "."


# **Define Features & Target for Training**

In [17]:
# Mode selector to choose between Live and Future prediction
mode_selector = widgets.ToggleButtons(
    options=["Live Delay Prediction", "Future Delay Prediction"],
    description="Select Mode:",
    style={'description_width': 'initial'}
)

# Containers for each mode
live_box = widgets.VBox()
future_box = widgets.VBox()
output_box = widgets.VBox([live_box])  # default mode is Live

def on_mode_change(change):
    if change['new'] == "Live Delay Prediction":
        output_box.children = [live_box]
    else:
        output_box.children = [future_box]

mode_selector.observe(on_mode_change, names='value')

display(mode_selector)
display(output_box)


ToggleButtons(description='Select Mode:', options=('Live Delay Prediction', 'Future Delay Prediction'), style=…

Error fetching live weather: HTTPSConnectionPool(host='api.open-meteo.com', port=443): Read timed out. (read timeout=5)


from matplotlib import pyplot as plt
_df_0['Mean Delay (min)'].plot(kind='hist', bins=20, title='Mean Delay (min)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Std Dev (min)'].plot(kind='hist', bins=20, title='Std Dev (min)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('Day').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['Day'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='Mean Delay (min)', y='Day', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_10['Day'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_10, x='Std Dev (min)', y='Day', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

Error fetching live weather: HTTPSConnectionPool(host='api.open-meteo.com', port=443): Read timed out. (read timeout=5)
Error fetching live weather: HTTPSConnectionPool(host='api.open-meteo.com', port=443): Read timed out. (read timeout=5)
Error fetching live weather: HTTPSConnectionPool(host='api.open-meteo.com', port=443): Read timed out. (read timeout=5)


# **Encode Categorical Data for Training**

In [18]:
# Widgets for live mode (temperature as read-only label)
temp_label = widgets.Label(value="Temperature (°C): --")
wind_slider = widgets.FloatSlider(min=0, max=50, step=0.5, description='Wind Speed (km/h):', value=0)
precip_slider = widgets.FloatSlider(min=0, max=10, step=0.1, description='Precipitation (mm):', value=0)
sensor_slider = widgets.FloatSlider(min=0.8, max=1.0, step=0.01, description='Sensor Reading:', value=0.85)

live_fetch_btn = widgets.Button(description="Get Right Now Delay")
live_msg_btn = widgets.Button(description="Get Message")
live_data_btn = widgets.Button(description="View Your Data")

live_voice_out = widgets.Output()
live_msg_out = widgets.Output()
live_data_out = widgets.Output()

# Global variables for current live prediction
current_explanation = ""
current_pred = 0.0
current_input_df = None

def live_predict(_=None):
    global current_explanation, current_pred, current_input_df
    # Fetch live weather data
    live_temp, live_wind, live_precip = fetch_live_weather()
    temp_label.value = f"Temperature (°C): {live_temp:.1f}"
    wind_slider.value = live_wind
    precip_slider.value = live_precip
    sensor_val = np.random.uniform(0.8, 1.0)
    sensor_slider.value = sensor_val
    comm = generate_sensor_alert(sensor_val)

    # Create input DataFrame for prediction
    current_input_df = pd.DataFrame({
        'Temperature': [live_temp],
        'WindSpeed': [live_wind],
        'Precipitation': [live_precip],
        'SensorReading': [sensor_val],
        'Crewed or Uncrewed': [0],  # 0 corresponds to 'Crewed'
        'Wind Direction': [0],      # 0 corresponds to 'N'
        'Condition': [0],           # 0 corresponds to 'Fair'
        'CommLog': [comm]
    })
    # Append space weather (for display only)
    current_input_df["SpaceWeather"] = fetch_space_weather()
    current_pred = model_pipeline.predict(current_input_df.drop(columns=['SpaceWeather']))[0]
    current_explanation = generate_delay_statement(current_input_df.iloc[0].to_dict(), current_pred)

    with live_voice_out:
        clear_output(wait=True)
        try:
            tts = gTTS(text=current_explanation, lang='en')
            tts.save("live_explanation.mp3")
            display(Audio("live_explanation.mp3", autoplay=True))
        except Exception as e:
            print("Voice error:", e)

def live_show_message(_):
    with live_msg_out:
        clear_output(wait=True)
        print("Prediction: {:.1f} minutes delay".format(current_pred))
        print("Explanation:", current_explanation)

def live_view_data(_):
    with live_data_out:
        clear_output(wait=True)
        disp_df = current_input_df.copy()
        disp_df['Crewed or Uncrewed'] = disp_df['Crewed or Uncrewed'].map({0:'Crewed', 1:'Uncrewed'})
        disp_df['Wind Direction'] = disp_df['Wind Direction'].map({0:'N', 1:'S', 2:'E', 3:'W', 4:'unknown'})
        disp_df['Condition'] = disp_df['Condition'].map({0:'Fair', 1:'Cloudy', 2:'Stormy'})
        print("----- Structured Data -----")
        display(disp_df[['Temperature', 'WindSpeed', 'Precipitation', 'SensorReading',
                         'Crewed or Uncrewed', 'Wind Direction', 'Condition']])
        print("----- Unstructured Data (CommLog) -----")
        display(disp_df[['CommLog']])
        print("----- Space Weather -----")
        display(disp_df[['SpaceWeather']])
        print("Predicted Delay: {:.1f} minutes".format(current_pred))

live_fetch_btn.on_click(live_predict)
live_msg_btn.on_click(live_show_message)
live_data_btn.on_click(live_view_data)

live_box.children = [
    widgets.HBox([live_fetch_btn, temp_label]),
    wind_slider, precip_slider, sensor_slider,
    live_voice_out,
    widgets.HBox([live_msg_btn, live_data_btn]),
    live_msg_out, live_data_out
]


# **Build a Multimodal Pipeline (Regression)**

In [19]:
# Widgets for future mode
future_days_inp = widgets.IntText(value=3, description="Days Ahead:")
future_btn = widgets.Button(description="Get Future Delay")
future_out = widgets.Output()

def generate_future_sample():
    base_temp, base_wind, base_precip = fetch_live_weather()
    fut_temp = np.clip(base_temp + np.random.normal(0, 1), 15, 35)
    fut_wind = np.clip(base_wind + np.random.normal(0, 2), 0, 50)
    fut_precip = np.clip(base_precip + np.random.normal(0, 0.5), 0, 10)
    fut_sensor = np.random.uniform(0.8, 1.0)
    fut_comm = generate_sensor_alert(fut_sensor)
    fut_space = fetch_space_weather()
    return {
        'Temperature': fut_temp,
        'WindSpeed': fut_wind,
        'Precipitation': fut_precip,
        'SensorReading': fut_sensor,
        'Crewed or Uncrewed': 0,
        'Wind Direction': 0,
        'Condition': 0,
        'CommLog': fut_comm,
        'SpaceWeather': fut_space
    }

def predict_future_delays(days=3, num_samples=30, delay_threshold=20):
    results = []
    for day in range(1, days+1):
        preds = []
        for _ in range(num_samples):
            sample = generate_future_sample()
            sample_df = pd.DataFrame([sample])
            pred = model_pipeline.predict(sample_df.drop(columns=['SpaceWeather']))[0]
            preds.append(pred)
        preds = np.array(preds)
        results.append({
            'Day': f"Day {day}",
            'Mean Delay (min)': round(preds.mean(), 2),
            'Std Dev (min)': round(preds.std(), 2),
            f"Prob(Delay > {delay_threshold} min) (%)": round(np.mean(preds > delay_threshold)*100, 2)
        })
    return pd.DataFrame(results)

def future_predict(_):
    days = future_days_inp.value
    fut_df = predict_future_delays(days=days, num_samples=30, delay_threshold=20)
    with future_out:
        clear_output(wait=True)
        print(f"Future Delay Predictions for Next {days} Day(s):")
        display(fut_df)
        summary = "Future predictions: " + ", ".join(
            [f"{row['Day']} mean delay {row['Mean Delay (min)']} minutes" for idx, row in fut_df.iterrows()]
        )
        try:
            tts = gTTS(text=summary, lang='en')
            tts.save("future_explanation.mp3")
            display(Audio("future_explanation.mp3", autoplay=True))
        except Exception as e:
            print("Voice error (future):", e)

future_btn.on_click(future_predict)
future_box.children = [widgets.HBox([future_days_inp, future_btn]), future_out]


In [20]:
# Display the main output box
display(output_box)

# Set default mode to Live and trigger initial live prediction.
mode_selector.value = "Live Delay Prediction"
live_predict()


Error fetching live weather: HTTPSConnectionPool(host='api.open-meteo.com', port=443): Read timed out. (read timeout=5)


from matplotlib import pyplot as plt
_df_11['Mean Delay (min)'].plot(kind='hist', bins=20, title='Mean Delay (min)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_12['Std Dev (min)'].plot(kind='hist', bins=20, title='Std Dev (min)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_13.groupby('Day').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_14.plot(kind='scatter', x='Mean Delay (min)', y='Std Dev (min)', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_19['Std Dev (min)'].plot(kind='line', figsize=(8, 4), title='Std Dev (min)')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['Day'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='Mean Delay (min)', y='Day', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['Day'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='Std Dev (min)', y='Day', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)